# Description
In this programming assignment, you are required to implement a contiguous sequential pattern mining algorithm and apply it on text data to mine potential phrase candidates.

## Input
The provided input file ("reviews_sample.txt") consists of 10,000 online reviews from Yelp users. The reviews have been stemmed (to remove the postfix of each word so words with similar semantics can have the same form), and most of the punctuation has been removed. Therefore, each line is basically a list of strings separated by spaces.

An example line is provided below:

`cold cheap beer good bar food good service looking great pittsburgh style fish 
sandwich place breading light fish plentiful good side home cut fry good 
grilled chicken salad steak soup day homemade lot special great place lunch 
bar snack beer`

## Task
You need to implement an algorithm to mine contiguous sequential patterns that are frequent in the input data. A contiguous sequential pattern is a sequence of items that frequently appears as a consecutive subsequence in a database of many sequences. For example, if the database is

`A,B,A,C
A,C,A,B,A,B
B,A,A,C,D`

and the minimum support is 2, then patterns like "A,B,A" or "A,C" are both frequent contiguous sequential patterns, while the pattern "A,A" is not a frequent contiguous sequential pattern because in the first two sequences the two A's are not consecutive to each other. Notice that it is still a frequent sequential pattern though.

Also, notice that multiple appearances of a subsequence in a single sequence record only counts once. For example, the pattern "A,B" appears 1 time in the first sequence and 2 times in the second, but its support should be calculated as 2, as there are only 2 records containing subsequence "A,B".


## SPADE Implementation
https://github.com/shahin/sequenceminer/blob/master/sequenceminer/miner.py

### Notes:
* Python 3 dictionary uses dict.items() instead of dict.iteritems()
    * https://stackoverflow.com/questions/30418481/error-dict-object-has-no-attribute-iteritems-when-trying-to-use-networkx
* Python 3, dict.keys() returns iterable but not indexable object
    * https://stackoverflow.com/questions/26394748/nltk-python-error-typeerror-dict-keys-object-is-not-subscriptable

In [36]:
import collections

class _KeyDefaultDict(collections.defaultdict):
    def __missing__(self,key):
        if self.default_factory is None:
            raise KeyError(key)
        else:
            ret = self[key] = self.default_factory(key)
            return ret

In [63]:
import collections
# from keydefaultdict import _KeyDefaultDict 

Event = collections.namedtuple('Event','sid eid')

class Element(object):
    '''An element of the set of all possible subsequences, and a description of
    where that element occurs in the input sequences.
    '''

    def __init__(self,seq,*events):

        self.seq = seq
        self.events = set()

        for event in events:
            self.events.add(event)

    def __ior__(self,other_element):
        '''Implements the assignment operator |= by returning an Element whose
        events attribute is a union of the events of both input Elements.
        '''

        self.events |= other_element.events
        return self

    def __repr__(self):

        return self.__dict__.__repr__()

    def __eq__(self,other):

        return (self.seq == other.seq and self.events == other.events)

      
def subset_to_support(elements,support_threshold):
    '''Given an IdList, return an IdList containing only those atoms which
    meet the support threshold.
    '''

    subsetted = _KeyDefaultDict(Element)

    for element_name,element in elements.items():
        support = len(set([event.sid for event in element.events]))
        if support >= support_threshold:
            subsetted[element_name] = element
                    
    return subsetted


def count_frequent_two_seq(elements,support_threshold):
    '''Given an IdList of atoms, return a dictionary of two-sequences as keys with
    the frequency of each two-sequence as the value.
    '''

    # Given an dictionary of Elements, convert it to a horizontal ID list in order to
    # count the frequency of each two-sequence of atoms.
    horizontal_db = {} 

    for element_name,element in elements.items():
                
        for event in element.events:

            if event.sid not in horizontal_db:
                 horizontal_db[event.sid] = []

            horizontal_db[event.sid].append((element_name,event.eid))

    # create counts using horizontal_db
    counts = collections.defaultdict(int)
    
    # for sid,seq in horizontal_db.iteritems():
    for sid,seq in horizontal_db.items():
        
        for event_index_i,event_i in enumerate(seq):
            for event_index_j,event_j in enumerate(seq[event_index_i+1:]):
                        
                if event_i[1] <= event_j[1]:
                    two_seq = event_i[0]+event_j[0]
                else:
                    two_seq = event_j[0]+event_i[0]

                counts[two_seq] += 1

    # this is followed by temporal joins between atoms in pairs, so
    # include only unique combinations
    # return {tuple(sorted(two_seq)) for two_seq,count in counts.iteritems() if count >= support_threshold}
    return {tuple(sorted(two_seq)) for two_seq,count in counts.items() if count >= support_threshold}
    
    
def temporal_join(element_i,element_j):
    '''Given two elements, return a dictionary of new elements indexed by
    their corresponding item sequences.
    '''

    join_results = _KeyDefaultDict(Element)
    
    for event_index_i,event_i in enumerate(element_i.events):
        for event_index_j,event_j in enumerate(element_j.events):
    
            if event_i.sid == event_j.sid:
                                        
                sid = event_i.sid
                superseqs = tuple()
                superseqs_events = tuple()
            
                # these two atoms occur in the same sequence
                # if they occur at different times (different eids), then
                # their combination atom has the later eid by Corollary 1 (Zaki 2001)
                if event_i.eid > event_j.eid:
                    superseq = element_j.seq + tuple(element_i.seq[-1])
                    superseq_event = Event(sid=sid,eid=event_i.eid)
                    join_results[superseq] |= Element(superseq,superseq_event)

                elif event_i.eid < event_j.eid:
                    superseq = element_i.seq + tuple(element_j.seq[-1])
                    superseq_event = Event(sid=sid,eid=event_j.eid)
                    join_results[superseq] |= Element(superseq,superseq_event)

                elif element_i.seq[-1] != element_j.seq[-1]:

                    superseq_event = Event(sid=sid,eid=event_j.eid)

                    # for coincident atoms, join the last element of one atom to the other
                    # ensure that the itemset is sorted
                    superseq_i = element_i.seq[:-1] + tuple([
                        ''.join(sorted(set(element_i.seq[-1] + element_j.seq[-1])))
                        ])
                    join_results[superseq_i] |= Element(superseq_i,superseq_event)

                    superseq_j = element_j.seq[:-1] + tuple([
                        ''.join(sorted(set(element_i.seq[-1] + element_j.seq[-1])))
                        ])

                    # if both resulting atoms are identical, only add it once
                    if superseq_j != superseq_i:
                        join_results[superseq_j] |= Element(superseq_j,superseq_event)
                
    return join_results


def enumerate_frequent_seq(elements,support_threshold):
    '''Recursively traverse the sequence lattice, generating frequent n+1-length
    sequences from n-length sequences provided in the id_list parameter.'''

    frequent_elements = _KeyDefaultDict(Element)

    
    for element_index_i,seq_i in enumerate(elements.keys()):

        frequent_elements_inner = _KeyDefaultDict(Element)
            
        # for element_index_j,seq_j in enumerate(elements.keys()[element_index_i+1:]):
        for element_index_j,seq_j in enumerate(list(elements.keys())[element_index_i+1:]):
            R = temporal_join(elements[seq_i],elements[seq_j])

            for seq,element in R.items():
                support = len(set([event.sid for event in element.events]))
                if support >= support_threshold:
                    frequent_elements_inner[seq] |= element


        for seq,element in frequent_elements_inner.items():
            frequent_elements[seq] |= element

        for seq,element in enumerate_frequent_seq(frequent_elements_inner,support_threshold).items():
            frequent_elements[seq] |= element

    return frequent_elements


def mine(sequences,support_threshold):
    '''SPADE (Zaki 2001) is performed in three distinct steps:
    1. Identify frequent single elements.
    2. Identify frequent two-element sequences.
    3. Identify all remaining sequences of three elements or more.
    '''

    # parse input sequences into individual item Elements
    elements = _KeyDefaultDict(Element) 

    for sid,eid,itemset in sequences:
        for item in itemset:
            elements[tuple(item)] |= Element(tuple(item),Event(sid=sid,eid=eid))

    # identify frequent single elements
    elements = subset_to_support(elements,support_threshold)

    # identify frequent two-element sequences using a horizontal database
    freq_elements_len_eq_2 = count_frequent_two_seq(elements,support_threshold)

    # generate ID lists for frequent two-element sequences discovered above
    elements_len_eq_2 = _KeyDefaultDict(Element)

    for two_seq in freq_elements_len_eq_2:

        R = temporal_join(elements[tuple(two_seq[0])],elements[tuple(two_seq[1])])

        for seq,element in R.items():
            support = len(set([event.sid for event in element.events]))
            if support >= support_threshold:
                elements_len_eq_2[seq] |= element

    # identify and generate ID lists for all remaining sequences
    freq = enumerate_frequent_seq(elements_len_eq_2,support_threshold)

    # collect all identified sequences of any length
    for seq,element in elements_len_eq_2.items():
        freq[seq] |= element

    for seq,element in elements.items():
        freq[seq] |= element

    # return frequent sequences
    return freq


def read_sequences(filename):
    '''Read sequences from a CSV.

    The CSV contains one line per sequence with columns defined as follows:
    - First column is a unique integer as sequence ID (sid)
    - Second column is a sequence-unique integer as event ID (eid)
    - Each remaining column contains an item as a character string with columns
      arranged in sequence order
    '''

    import csv

    sequences = []

    with open(filename) as f:
        seqreader = csv.reader(f,delimiter=',')
        for seqline in seqreader:
            sequences.append(
                    tuple([ seqline[0],seqline[1],tuple(seqline[2:]) ])
                    )

    return sequences

## Reformat transactions input and output
1. read text file into list of lists
2. generate SID, EID, and Item (as character string)
    * Final sequences should be a list of tuples (SID, EID, and Item)
3. convert output into sequence; support - Pandas?

In [110]:
# Step 1. read "reviews_samples.txt" into list of lists
# https://stackoverflow.com/questions/18448847/import-txt-file-and-having-each-line-as-a-list
transactions = []
with open('reviews_sample.txt', 'rt') as f:
    for line in f:
        transactions.append(line.strip().split(' '))
print(type(transactions))
print(type(transactions[0]))
print(len(transactions))

<class 'list'>
<class 'list'>
10000


In [117]:
# Test
transactions = [
    ['A','B','A','C'],
    ['A','C','A','B','A','B'],
    ['B','A','A','C','D']]
                

In [118]:
# Step 2. generate SID, EID, and Item (as character string)
# Each tuple is (SID, EID, Item)
# Add tuple to list (sequences)
sequences = []
for line_num, line in enumerate(transactions): # SID corresponds to line_num + 1
    for item_num, item in enumerate(line): # EID corresponds to item_num + 1
        tup = (line_num+1, item_num+1, item)
        sequences.append(tup)        

In [119]:
for entry in sequences:
    print(entry)

(1, 1, 'A')
(1, 2, 'B')
(1, 3, 'A')
(1, 4, 'C')
(2, 1, 'A')
(2, 2, 'C')
(2, 3, 'A')
(2, 4, 'B')
(2, 5, 'A')
(2, 6, 'B')
(3, 1, 'B')
(3, 2, 'A')
(3, 3, 'A')
(3, 4, 'C')
(3, 5, 'D')


In [101]:
# Test mine()
import pprint as pp

frequent_sequences = mine(sequences, 2) # use absolute support
print(type(frequent_sequences))

cnt = 0

for k, v in frequent_sequences.items():
    print(len(v.events)) # support is num of events
    print(k) # key is sequence represented as tuple
    cnt+=1
print(cnt)

<class '__main__._KeyDefaultDict'>
2
('A', 'A', 'C')
2
('B', 'A', 'C')
2
('A', 'B', 'A')
3
('A', 'C')
4
('A', 'A')
2
('B', 'C')
4
('B', 'A')
3
('A', 'B')
7
('A',)
4
('B',)
3
('C',)
11


In [103]:
# Step 3a. Add to list of lists
all_frequent_sequences = []

for k, v in frequent_sequences.items():
    all_frequent_sequences.append([len(v.events), k])
    #print(len(v.events)) # support is num of events
    #print(k) # key is sequence represented as tuple
    cnt+=1
print(cnt)
all_frequent_sequences

33


[[2, ('A', 'A', 'C')],
 [2, ('B', 'A', 'C')],
 [2, ('A', 'B', 'A')],
 [3, ('A', 'C')],
 [4, ('A', 'A')],
 [2, ('B', 'C')],
 [4, ('B', 'A')],
 [3, ('A', 'B')],
 [7, ('A',)],
 [4, ('B',)],
 [3, ('C',)]]

In [106]:
# Step 3b. Convert to pandas dataframe
import pandas as pd

# support is int, and sequence is tuple
all_frequent_sequences_df = pd.DataFrame(all_frequent_sequences, columns=['support', 'sequence'])
all_frequent_sequences_df

,support,sequence
0,2,"(A, A, C)"
1,2,"(B, A, C)"
2,2,"(A, B, A)"
3,3,"(A, C)"
4,4,"(A, A)"
5,2,"(B, C)"
6,4,"(B, A)"
7,3,"(A, B)"
8,7,"(A,)"
9,4,"(B,)"


## Phrase Mining (Generate n-grams)

## Format Results + Write to File

In [7]:
seq_pattern = []
# print Formatting
for seq_dict in result:
    for k, v in seq_dict.items():
        seq_pattern = [x for x in k]
        print(str(v) + ':' + ';'.join(map(str, seq_pattern)) + '\n')

3:A

3:B

3:C

2:A;B

3:A;C

3:B;A

2:A;B;A



In [8]:
seq_pattern = []
with open('output/patterns_test.txt', 'wt') as f:
    for seq_dict in result:
        for k, v in seq_dict.items():
            seq_pattern = [x for x in k]
            f.write(str(v) + ':' + ';'.join(map(str, seq_pattern)) + '\n')

## Output
Please set the relative minimum support to 0.01 and run it on the given text file. In other words, you need to extract all the frequent contiguous sequential patterns that have an absolute support no smaller than 100.

Please write all the frequent contiguous sequential patterns along with their absolute supports into a text file named "patterns.txt". Every line corresponds to exactly one pattern you found and should be in the following format:

support:item_1;item_2;item_3

For example, suppose the phrase "parking lot" has an absolute support 133, then the line corresponding to this frequent contiguous sequential pattern in "patterns.txt" should be:

133:parking;lot

Notice that the order does matter in sequential pattern mining. That is to say,

133:lot;parking

may be graded as incorrect.

## Run Spade to Generate Frequent, Sequential Patterns

In [120]:
# Step 1. read "reviews_samples.txt" into list of lists
# https://stackoverflow.com/questions/18448847/import-txt-file-and-having-each-line-as-a-list
transactions = []
with open('reviews_sample.txt', 'rt') as f:
    for line in f:
        transactions.append(line.strip().split(' '))
print(type(transactions))
print(type(transactions[0]))
print(len(transactions))

<class 'list'>
<class 'list'>
10000


In [136]:
transasctions = transasctions[0:10] # test on 100

In [137]:
# Step 2. generate SID, EID, and Item (as character string)
# Each tuple is (SID, EID, Item)
# Add tuple to list (sequences)
sequences = []
for line_num, line in enumerate(transactions): # SID corresponds to line_num + 1
    for item_num, item in enumerate(line): # EID corresponds to item_num + 1
        tup = (line_num+1, item_num+1, item)
        sequences.append(tup)        

### Perform SPADE mine()

In [138]:
%%time
frequent_sequences = mine(sequences, 100) # use absolute support no smaller than 100

KeyboardInterrupt: 

In [ ]:
# Step 3a. Add to list of lists
all_frequent_sequences = []

for k, v in frequent_sequences.items():
    all_frequent_sequences.append([len(v.events), k])
    #print(len(v.events)) # support is num of events
    #print(k) # key is sequence represented as tuple
    cnt+=1
print(cnt)
all_frequent_sequences.head()

In [ ]:
# Step 3b. Convert to pandas dataframe
import pandas as pd

# support is int, and sequence is tuple
all_frequent_sequences_df = pd.DataFrame(all_frequent_sequences, columns=['support', 'sequence'])
all_frequent_sequences_df.head()

In [ ]:
all_frequent_sequences_df.to_csv("output/SPADE_output.csv")